In [9]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
from pytz import timezone

In [10]:
__TIMEZONE = timezone('Europe/London')
days = 1/24

In [11]:
_DB_URL = "/Users/kws/dropbox/iot/eeepc/iot-log.db"

In [12]:
start_date = datetime.utcnow() - timedelta(days=days)
start = start_date.strftime('%Y-%m-%dT%H:%M:%S')

with sqlite3.connect(_DB_URL) as conn:
    df = pd.read_sql_query(
        "select l.date,l.id,s.name,l.type,l.value,s.sort_order from log as l"
        "  left join sensors as s on l.id = s.id "
        "  where l.date>=? and l.type=? order by l.date",
        conn, params=[start, 'presence'])
    
df['date'] = pd.to_datetime(df.date)
df['date'] = df['date'].dt.tz_localize(timezone('UTC'))
df['date'] = df['date'].dt.tz_convert(__TIMEZONE)
orig = df
df

,date,id,name,type,value,sort_order
0,2020-04-18 14:41:41+01:00,47,Kitchen,presence,0,1
1,2020-04-18 14:42:15+01:00,53,Bedroom,presence,0,2
2,2020-04-18 14:42:30+01:00,47,Kitchen,presence,1,1
3,2020-04-18 14:42:31+01:00,42,Garage,presence,1,3
4,2020-04-18 14:42:48+01:00,83,Garden,presence,1,5
...,...,...,...,...,...,...
162,2020-04-18 15:27:33+01:00,47,Kitchen,presence,0,1
163,2020-04-18 15:28:24+01:00,53,Bedroom,presence,1,2
164,2020-04-18 15:28:38+01:00,53,Bedroom,presence,0,2
165,2020-04-18 15:36:02+01:00,53,Bedroom,presence,1,2


In [43]:
df = orig.copy()
periods = 30
time_window = days / periods

start_date_tz = timezone('UTC').localize(start_date)
# start_date_tz = start_date.astimezone(__TIMEZONE)

df = df.sort_values(["id","date"]).reset_index(drop=True)
df.loc[df.value == '1', 'start'] = df.date
df.loc[df.value == '0', 'end'] = df.date

# Fill first and last of each sensor with the start and end of dataframe
df.loc[(df.start.isnull()) & (df.id != df.id.shift(1)), 'start'] = start_date_tz
df.loc[(df.end.isnull()) & (df.id != df.id.shift(-1)), 'end'] = start_date_tz + timedelta(days=days)

df[['end']] = df[['end']].fillna(method='bfill')
df[['start']] = df[['start']].fillna(method='ffill')

df.head(50).tail(15)

,date,id,name,type,value,sort_order,start,end
35,2020-04-18 15:22:29+01:00,42,Garage,presence,1,3,2020-04-18 15:22:29+01:00,2020-04-18 15:24:00+01:00
36,2020-04-18 15:24:00+01:00,42,Garage,presence,0,3,2020-04-18 15:22:29+01:00,2020-04-18 15:24:00+01:00
37,2020-04-18 15:24:01+01:00,42,Garage,presence,1,3,2020-04-18 15:24:01+01:00,2020-04-18 15:25:02+01:00
38,2020-04-18 15:24:44+01:00,42,Garage,presence,1,3,2020-04-18 15:24:44+01:00,2020-04-18 15:25:02+01:00
39,2020-04-18 15:25:02+01:00,42,Garage,presence,0,3,2020-04-18 15:24:44+01:00,2020-04-18 15:25:02+01:00
40,2020-04-18 15:25:10+01:00,42,Garage,presence,1,3,2020-04-18 15:25:10+01:00,2020-04-18 15:25:24+01:00
41,2020-04-18 15:25:24+01:00,42,Garage,presence,0,3,2020-04-18 15:25:10+01:00,2020-04-18 15:25:24+01:00
42,2020-04-18 14:41:41+01:00,47,Kitchen,presence,0,1,2020-04-18 13:41:38.328362+00:00,2020-04-18 14:41:41+01:00
43,2020-04-18 14:42:30+01:00,47,Kitchen,presence,1,1,2020-04-18 14:42:30+01:00,2020-04-18 14:43:14+01:00
44,2020-04-18 14:43:14+01:00,47,Kitchen,presence,0,1,2020-04-18 14:42:30+01:00,2020-04-18 14:43:14+01:00


In [59]:
df['delta'] = df['start'] + timedelta(days=time_window)
df['delta_prev'] = df.delta.shift(1)
next_sensor = df.id != df.id.shift(1)
df.loc[next_sensor, 'delta'] = df.end
df.loc[next_sensor, 'delta_prev'] = df.start - timedelta(seconds=1)
df.head(50).tail(15)

,date,id,name,type,value,sort_order,start,end,delta,delta_prev,group_id,delta_end
35,2020-04-18 15:22:29+01:00,42,Garage,presence,1,3,2020-04-18 15:22:29+01:00,2020-04-18 15:24:00+01:00,2020-04-18 15:24:29+01:00,2020-04-18 15:23:57+01:00,33.0,NaN
36,2020-04-18 15:24:00+01:00,42,Garage,presence,0,3,2020-04-18 15:22:29+01:00,2020-04-18 15:24:00+01:00,2020-04-18 15:24:29+01:00,2020-04-18 15:24:29+01:00,33.0,NaN
37,2020-04-18 15:24:01+01:00,42,Garage,presence,1,3,2020-04-18 15:24:01+01:00,2020-04-18 15:25:02+01:00,2020-04-18 15:26:01+01:00,2020-04-18 15:24:29+01:00,33.0,NaN
38,2020-04-18 15:24:44+01:00,42,Garage,presence,1,3,2020-04-18 15:24:44+01:00,2020-04-18 15:25:02+01:00,2020-04-18 15:26:44+01:00,2020-04-18 15:26:01+01:00,33.0,NaN
39,2020-04-18 15:25:02+01:00,42,Garage,presence,0,3,2020-04-18 15:24:44+01:00,2020-04-18 15:25:02+01:00,2020-04-18 15:26:44+01:00,2020-04-18 15:26:44+01:00,33.0,NaN
40,2020-04-18 15:25:10+01:00,42,Garage,presence,1,3,2020-04-18 15:25:10+01:00,2020-04-18 15:25:24+01:00,2020-04-18 15:27:10+01:00,2020-04-18 15:26:44+01:00,33.0,NaN
41,2020-04-18 15:25:24+01:00,42,Garage,presence,0,3,2020-04-18 15:25:10+01:00,2020-04-18 15:25:24+01:00,2020-04-18 15:27:10+01:00,2020-04-18 15:27:10+01:00,33.0,NaN
42,2020-04-18 14:41:41+01:00,47,Kitchen,presence,0,1,2020-04-18 13:41:38.328362+00:00,2020-04-18 14:41:41+01:00,2020-04-18 14:41:41+01:00,2020-04-18 13:41:37.328362+00:00,33.0,dpdp
43,2020-04-18 14:42:30+01:00,47,Kitchen,presence,1,1,2020-04-18 14:42:30+01:00,2020-04-18 14:43:14+01:00,2020-04-18 14:44:30+01:00,2020-04-18 13:43:38.328362+00:00,33.0,NaN
44,2020-04-18 14:43:14+01:00,47,Kitchen,presence,0,1,2020-04-18 14:42:30+01:00,2020-04-18 14:43:14+01:00,2020-04-18 14:44:30+01:00,2020-04-18 14:44:30+01:00,33.0,NaN


In [60]:
df['group_id'] = df.index
df.loc[df.start <= df.delta_prev, 'group_id'] = np.nan
df['group_id'] = df['group_id'].fillna(method='ffill')
df.head(50).tail(15)

,date,id,name,type,value,sort_order,start,end,delta,delta_prev,group_id,delta_end
35,2020-04-18 15:22:29+01:00,42,Garage,presence,1,3,2020-04-18 15:22:29+01:00,2020-04-18 15:24:00+01:00,2020-04-18 15:24:29+01:00,2020-04-18 15:23:57+01:00,33.0,NaN
36,2020-04-18 15:24:00+01:00,42,Garage,presence,0,3,2020-04-18 15:22:29+01:00,2020-04-18 15:24:00+01:00,2020-04-18 15:24:29+01:00,2020-04-18 15:24:29+01:00,33.0,NaN
37,2020-04-18 15:24:01+01:00,42,Garage,presence,1,3,2020-04-18 15:24:01+01:00,2020-04-18 15:25:02+01:00,2020-04-18 15:26:01+01:00,2020-04-18 15:24:29+01:00,33.0,NaN
38,2020-04-18 15:24:44+01:00,42,Garage,presence,1,3,2020-04-18 15:24:44+01:00,2020-04-18 15:25:02+01:00,2020-04-18 15:26:44+01:00,2020-04-18 15:26:01+01:00,33.0,NaN
39,2020-04-18 15:25:02+01:00,42,Garage,presence,0,3,2020-04-18 15:24:44+01:00,2020-04-18 15:25:02+01:00,2020-04-18 15:26:44+01:00,2020-04-18 15:26:44+01:00,33.0,NaN
40,2020-04-18 15:25:10+01:00,42,Garage,presence,1,3,2020-04-18 15:25:10+01:00,2020-04-18 15:25:24+01:00,2020-04-18 15:27:10+01:00,2020-04-18 15:26:44+01:00,33.0,NaN
41,2020-04-18 15:25:24+01:00,42,Garage,presence,0,3,2020-04-18 15:25:10+01:00,2020-04-18 15:25:24+01:00,2020-04-18 15:27:10+01:00,2020-04-18 15:27:10+01:00,33.0,NaN
42,2020-04-18 14:41:41+01:00,47,Kitchen,presence,0,1,2020-04-18 13:41:38.328362+00:00,2020-04-18 14:41:41+01:00,2020-04-18 14:41:41+01:00,2020-04-18 13:41:37.328362+00:00,42.0,dpdp
43,2020-04-18 14:42:30+01:00,47,Kitchen,presence,1,1,2020-04-18 14:42:30+01:00,2020-04-18 14:43:14+01:00,2020-04-18 14:44:30+01:00,2020-04-18 13:43:38.328362+00:00,42.0,NaN
44,2020-04-18 14:43:14+01:00,47,Kitchen,presence,0,1,2020-04-18 14:42:30+01:00,2020-04-18 14:43:14+01:00,2020-04-18 14:44:30+01:00,2020-04-18 14:44:30+01:00,42.0,NaN


In [61]:
df.groupby('group_id').agg({'name': 'first', 'start': min, 'end':max, 'sort_order': 'first'}).reset_index()

,group_id,name,start,end,sort_order
0,0.0,Garage,2020-04-18 14:42:31+01:00,2020-04-18 14:50:49+01:00,3
1,14.0,Garage,2020-04-18 14:54:28+01:00,2020-04-18 14:56:46+01:00,3
2,20.0,Garage,2020-04-18 15:01:21+01:00,2020-04-18 15:03:21+01:00,3
3,23.0,Garage,2020-04-18 15:06:34+01:00,2020-04-18 15:07:47+01:00,3
4,27.0,Garage,2020-04-18 15:16:24+01:00,2020-04-18 15:18:22+01:00,3
5,29.0,Garage,2020-04-18 15:18:27+01:00,2020-04-18 15:19:26+01:00,3
6,33.0,Garage,2020-04-18 15:21:57+01:00,2020-04-18 15:25:24+01:00,3
7,42.0,Kitchen,2020-04-18 13:41:38.328362+00:00,2020-04-18 14:50:55+01:00,1
8,67.0,Kitchen,2020-04-18 14:54:25+01:00,2020-04-18 14:59:04+01:00,1
9,72.0,Kitchen,2020-04-18 14:59:11+01:00,2020-04-18 15:04:20+01:00,1
